## 딕셔너리와 해시테이블

해시테이블은 희소배열(중간중간에 빈 값이 들어간 배열)이다. 딕셔너리의 해시테이블에는 각 항목 별로 버킷이 있고 버킷에는 키에 대한 참조와 항목의 값에 대한 참조가 들어간다.
모든 버킷의 크기가 동일하므로 오프셋을 계산해서 각 버킷에 바로 접근할 수 있다.
해시테이블의 항목이 많아지면 더 넓은 공간에 복사해서 버킷 공간을 확보한다.
해시테이블 안에 항목을 넣을 때 먼저 항목 키의 해시값을 계산한다. (`hash()` built-in 함수를 사용한다.)<br>
https://docs.python.org/3/reference/datamodel.html#object.__hash__

built-in 자료형은 `hash()`를 호출하고 사용자 정의 자료형의 객체는 `__hash__()`메서드를 호출한다. 두 객체가 동일하면 해시값도 동일해야 한다.
예를 들어 정수 1과 실수 1의 표현 형태는 다르지만 `1 == 1.0`은 참이므로 `hash(1) == hash(1.0)`도 참이어야 한다.

해시 테이블의 효율성을 높이려면 해시값은 인덱싱 공간에 최대한 고르게 퍼져야 한다. 즉, 비슷하지만 동일하지는 않은 객체들의 해시값은 상당히 달라야 한다.

In [20]:
print(hash(1))
print(hash(1.0))
print(hash(1.0001))
print(hash(1.0002))

1
1
230584300921345
461168601842689


`my_dic[my_key]`에서 값을 가져오기 위해 파이썬은 `__hash__(my_key)`를 호출하여 `my_key`의 해시값을 가져오고 해시값의 최하위 비트를 해시테이블 안의
버킷에 대한 오프셋으로 사용한다. 사용하는 비트 수는 현재 테이블 크기에 따라 달라진다.
찾아낸 버킷이 비어있으면 `KeyError`를 발생시키고 그렇지 않으면 버킷에 들어있는 항목인 `(found_key(키에 대한 참조): found_value(항목의 값에 대한 참조))` 쌍을 검사해서
`my_key == found_key` 가 참인지 검사한다. 참이면 항목을 찾은 것이므로 `found_value`를 리턴한다.

In [24]:
from dis import dis

my_dict = {
    10: 'a',
    11: 'b',
    12: 'c'
}
dis('my_dict[10]')
dis('my_dict[12]')  
# BINRAY_SUBSCR implements TOS = TOS1[TOS].
# https://docs.python.org/2.4/lib/bytecodes.html

  1           0 LOAD_NAME                0 (my_dict)
              2 LOAD_CONST               0 (10)
              4 BINARY_SUBSCR
              6 RETURN_VALUE
  1           0 LOAD_NAME                0 (my_dict)
              2 LOAD_CONST               0 (12)
              4 BINARY_SUBSCR
              6 RETURN_VALUE


`found_key`와 `my_key`가 다른 경우는 해시 충돌(hash collision)이 발생했기 때문이다.
해시 충돌은 서로 다른 키가 같은 해시값을 가질 때 발생하고 주로 해싱함수가 임의의 키를 적은 수의 비트로 매핑하는 것이 원인이다.
해시 충돌을 해결하기 위해 해시값으로부터 가져온 비트로 버킷의 위치를 다시 조회한다. 위치를 조회했을 때 버킷이 비어있으면 `KeyError`를
발생시키고 그렇지 않으면 `found_value`를 반환한다.<br>
딕셔너리에 키/값을 추가하거나 수정하는 과정도 동일하다. 키/값을 추가하는 경우는 빈 버킷이 발견되었을 때 키/값을 버킷에 삽입하고, 수정하는 경우는 `found_key`를 가진 버킷을 찾으면 `found_value`를 갱신한다.